In [1]:
import dgl
from dgl.data import BAShapeDataset
import torch
import torch_geometric
import torch_geometric.transforms as T

In [2]:
import argparse
import time
import torch
import easydict
import torch_geometric
import random
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
# This code works in Python 3.10.6
import matplotlib.pyplot as plt
import networkx as nx
import torch
import torch_geometric.utils
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.datasets.dblp import DBLP
from torch_geometric.nn import GCNConv
import time
from torch_geometric.logging import log
import os
from collections import Counter

In [4]:
dataset = BAShapeDataset()

Done loading data from cached files.


In [5]:
g = dataset[0]

In [6]:
g

Graph(num_nodes=700, num_edges=2055,
      ndata_schemes={'feat': Scheme(shape=(1,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'__orig__': Scheme(shape=(), dtype=torch.int64)})

In [7]:
label = g.ndata['label']
len(label)

700

In [8]:
n_nodes = g.number_of_nodes()
n_nodes

700

In [9]:
#Download file BA_shapes.pkl from the dataset in https://github.com/Graph-and-Geometric-Learning/D4Explainer. BA_shapes.pkl is required for the train/val/test splits.

In [10]:
import pickle
with open('BA_shapes.pkl', 'rb') as fin:
    adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, edge_label_matrix  = pickle.load(fin)

In [11]:
g.ndata["train_mask"] = torch.tensor(train_mask)
g.ndata["val_mask"] = torch.tensor(val_mask)
g.ndata["test_mask"] = torch.tensor(test_mask)

In [12]:
g

Graph(num_nodes=700, num_edges=2055,
      ndata_schemes={'feat': Scheme(shape=(1,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'__orig__': Scheme(shape=(), dtype=torch.int64)})

In [13]:
data = torch_geometric.utils.from_dgl(g)
data

Data(edge_index=[2, 2055], feat=[700, 1], label=[700], train_mask=[700], val_mask=[700], test_mask=[700], __orig__=[2055])

In [14]:
data.x = data.pop('feat')
data.y = data.pop('label')
data.pop('__orig__')

tensor([2008, 1840, 1812,  ..., 1542, 1538, 1543])

In [15]:
x = torch.tensor([1.0,0.0,0.0,0.0])
data.x = x.repeat(data.x.shape[0],1)

In [16]:
data.x

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]])

In [18]:
data.x.shape

torch.Size([700, 4])

In [19]:
data

Data(edge_index=[2, 2055], train_mask=[700], val_mask=[700], test_mask=[700], x=[700, 4], y=[700])

In [20]:
torch.save(data, '../data/BA_shapes/processed/data.pt')

In [21]:
torch.unique(data.y)

tensor([0, 1, 2, 3])

In [22]:
Counter(data.y.tolist())

Counter({0: 300, 2: 160, 1: 160, 3: 80})

In [23]:
data.has_isolated_nodes()

False

In [24]:
data.has_self_loops()

False